<a href="https://colab.research.google.com/github/zrghassabi/LLM/blob/main/Chapter3_Solution%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# English-German Translation using FLAN-T5 and WMT16 Dataset

## Introduction
In this notebook, we will use the FLAN-T5 model to perform translation from English to German using the WMT16 dataset. We will preprocess the data, fine-tune the model, and evaluate its performance using the BLEU metric.

In [ ]:
!pip install transformers tensorflow datasets nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

## Choose and Preprocess Dataset

In [ ]:
from datasets import load_dataset

# Load the WMT16 English-German dataset
dataset = load_dataset('wmt16', 'de-en')

# Display an example
print(dataset['train'][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
model = TFAutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Preprocess the dataset for input into the model
def preprocess_data(examples):
    inputs = [f'Translate English to German: {example["en"]}' for example in examples['translation']]
    targets = [example['de'] for example in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length', return_tensors='tf').input_ids
    model_inputs['labels'] = labels
    decoder_inputs = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["decoder_input_ids"] = decoder_inputs["input_ids"]
    return model_inputs


train_dataset = dataset['train'].select(range(30000)).map(preprocess_data, batched=True)
test_dataset = dataset['test'].select(range(2000)).map(preprocess_data, batched=True)

train_dataset = train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=64,
    collate_fn=None
)

test_dataset = test_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=64,
    collate_fn=None
)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:410: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 247577856 (944.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.get_layer('shared').trainable = False
model.get_layer('encoder').trainable = False
model.get_layer('decoder').trainable = False

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 24674304 (94.12 MB)
Non-trainable params: 222903552 (850.31 MB)
_________________________________________________________________


## Train the Model

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
469/469 [==============================] - 300s 442ms/step - loss: 38.5995 - val_loss: 41.4301
Epoch 2/3
469/469 [==============================] - 180s 383ms/step - loss: 37.7107 - val_loss: 40.5200
Epoch 3/3
469/469 [==============================] - 180s 383ms/step - loss: 36.8942 - val_loss: 39.6269


## Evaluate the Model

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

# Function to calculate BLEU score
def calculate_bleu(reference, hypothesis):
    reference_tokens = [nltk.word_tokenize(reference)]
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    bleu_score = sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=SmoothingFunction().method4)
    return bleu_score

# Function to translate and evaluate
def translate_and_evaluate(dataset):
    bleu_scores = []
    batch = next(iter(dataset))
    inputs = batch[0]['input_ids']
    references = batch[1]
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    for i in range(len(inputs)):
        reference = tokenizer.decode(references[i], skip_special_tokens=True)
        hypothesis = tokenizer.decode(outputs[i], skip_special_tokens=True)
        bleu_score = calculate_bleu(reference, hypothesis)
        bleu_scores.append(bleu_score)
    return sum(bleu_scores) / len(bleu_scores)

# Evaluate on the validation set
average_bleu_score = translate_and_evaluate(test_dataset)
print(f'Average BLEU score on validation set: {average_bleu_score}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Average BLEU score on validation set: 0.12046652716839085


## Conclusion
In this notebook, we used the FLAN-T5 model to perform translation from English to German using the WMT16 dataset. We preprocessed the dataset, fine-tuned the model, and evaluated its performance using the BLEU metric. The results demonstrate the effectiveness of the FLAN-T5 model for translation tasks.